# Binary classification : logistic regression is best

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Preprocessing

In [8]:
news_dataset=pd.read_csv("/content/fake_news_prediction.csv")

In [9]:
news_dataset.shape

(20800, 5)

In [11]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
news_dataset.isnull().sum().sum()

2554

In [13]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
# replacing null values with empty strings. as only test,author and title me missing hai. So null string se fill kr skte hain
news_dataset=news_dataset.fillna('')

In [15]:
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [17]:
print(news_dataset['content']) # we will be only osing content and label for prediction. Nothing else

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [18]:
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']
# ye X farji hai, galti se yhan use kr liye hain

In [19]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content) :
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) # alphabes ke alava numbers and otheer things ko remove krega content column se
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # all stopwords tko content column se hata dega
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [24]:
news_dataset['content']=news_dataset['content'].apply(stemming)
 # content series ke sabhi elements pr stemming function apply ho jayega

.apply() function in pandas:

The primary purpose of the .apply() function is to transform data in a Pandas Series or DataFrame by applying a function to each element.

import pandas as pd

# Create a Pandas Series
data = [1, 2, 3, 4, 5]

series = pd.Series(data)

# Define a function
def square(x):
    return x ** 2

# Apply the function to each element of the Series

result = series.apply(square)

print(result) : series ke sare elements ka square print hoga


In [25]:
# separating the data and labels
X=news_dataset['content'].values
T=news_dataset['label'].values

In [26]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [27]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [29]:
# converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer()

In [30]:
X=vectorizer.transform(X)

In [31]:
print(X)

  (0, 15553)	0.28485063562728646
  (0, 13348)	0.2565896679337957
  (0, 8822)	0.3635963806326075
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 6930)	0.21874169089359144
  (0, 4917)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16656)	0.3025156488372128
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  (2, 5326)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12998)	0.2483705036831893
  (20797, 12227)	0.27276402145717243
  (20797, 12022)	0.24790022252744132
  (20797, 10216)	0.0804189541935242
  (20797, 9500)	0.17463635692029988
  (20797, 9431)	0.29394

In [43]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

# training the model : logistic regression

In [44]:
model=LogisticRegression()

In [45]:
model.fit(x_train,y_train)

LogisticRegression()

# accuracy score on the training data


In [46]:
x_train_prediction=model.predict(x_train) # values predicted by our model, but the shown values are same as what he has already seen.
training_data_accuracy=accuracy_score(x_train_prediction,y_train) # y train= actual values. accuracy score is calculated based on comparison between x_train_prediction and y_train
training_data_accuracy

0.9866586538461538

In [47]:
# accuraacy on test data
x_test_prediction=model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_prediction,y_test) # by the comparison betweeen, predicted values from model and actual values(y_test)
test_data_accuracy

0.9790865384615385

## Making a predictive system : news is real or fake

In [48]:
x_new = x_test[3]
prediction = model.predict(x_new)
print(prediction)

if(prediction==0):
  print("The news is real")
else:
  print("The news is fake")


[0]
The news is real


In [55]:
print(x_test[3])

  (0, 14504)	0.23474748316072222
  (0, 12572)	0.212825991610646
  (0, 11773)	0.2548652041974303
  (0, 11140)	0.3244395154569377
  (0, 11113)	0.33219630487929364
  (0, 7589)	0.18859131768467946
  (0, 6795)	0.2745539220781882
  (0, 5465)	0.29122456917649464
  (0, 3749)	0.5176845437180357
  (0, 2783)	0.22720201008618632
  (0, 2491)	0.19691105758309724
  (0, 2383)	0.23077041549913466
  (0, 1877)	0.10780435789975806


In [56]:
print(y_test) # 3rd =0, as expected => model predicting correctly

20536    1
13735    0
2973     1
12477    0
4490     0
        ..
13698    0
9973     1
11030    1
20399    1
7899     0
Name: label, Length: 4160, dtype: int64
